In [ ]:
%%sql

CREATE VIEW view_economic_data AS
SELECT
    dd.year,
    dc.country,
    dc.region,
    di.indicator_name,
    di.category,
    fei.value
FROM fact_economic_indicator AS fei
INNER JOIN dim_date AS dd
    ON fei.date_key = dd.date_key
INNER JOIN dim_indicator AS di
    ON fei.indicator_key = di.indicator_key
INNER JOIN dim_country AS dc
    ON fei.country_key = dc.country_key

In [ ]:
%%sql

CREATE VIEW view_yoy_growth AS 
SELECT
    dd.year,
    dc.country_name,
    di.indicator_name,
    LAG(f.value) OVER (
        PARTITION BY f.country_key, f.indicator_key 
        ORDER BY d.year
    ) as previous_value,
    ((f.value - LAG(f.value) OVER (
        PARTITION BY f.country_key, f.indicator_key 
        ORDER BY d.year
    )) / NULLIF(LAG(f.value) OVER (
        PARTITION BY f.country_key, f.indicator_key 
        ORDER BY d.year
    ), 0)) * 100 as yoy_growth_pct
FROM fact_economic_indicator AS fei
INNER JOIN dim_date AS dd
    ON fei.date_key = dd.date_key
INNER JOIN dim_indicator AS di
    ON fei.indicator_key = di.indicator_key
INNER JOIN dim_country AS dc
    ON fei.country_key = dc.country_key